In [1]:
import pandas as pd
import numpy as np
import random as rand
import importlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
raw = pd.read_csv("yellow-taxis/1january.csv")

In [3]:
# delete unused columns
# del raw['trip_distance']
del raw['passenger_count']
del raw['fare_amount']
del raw['tolls_amount']
del raw['taxes_amount']
del raw['tip_amount']
# del raw['payment_amount']
del raw['payment_type']

In [4]:
# convert pickup_dtatetime to datetime
raw["pickup_datetime"] = pd.to_datetime(raw["pickup_datetime"])
raw["dropoff_datetime"] = pd.to_datetime(raw["dropoff_datetime"])
raw = raw[(raw["pickup_longitude"] > -80) & (raw["pickup_longitude"] < -60)
            & (raw["dropoff_longitude"] > -80) & (raw["dropoff_longitude"] < -60)
            & (raw["pickup_latitude"] > 30) & (raw["pickup_latitude"] < 50)
            & (raw["dropoff_latitude"] > 30) & (raw["dropoff_latitude"] < 50)]
shuffle(raw)
raw

NameError: name 'shuffle' is not defined

In [ ]:
# convert pickup_dtatetime to datetime
raw["pickup_datetime"] = pd.to_datetime(raw["pickup_datetime"])
raw["dropoff_datetime"] = pd.to_datetime(raw["dropoff_datetime"])

In [ ]:
#make sure types are okay
print(list(raw.columns.values))
print([raw[i].dtype for i in list(raw.columns.values)])

In [6]:
#convert it to a numpy matrix
#np_raw = raw.as_matrix()

In [7]:
#print(np_raw.dtype)
print(raw.ix[0])
print(len(raw))

pickup_datetime      2016-01-01 00:00:00
pickup_latitude                  40.7347
pickup_longitude                -73.9904
trip_distance                        1.1
dropoff_datetime     2016-01-01 00:00:00
dropoff_latitude                 40.7324
dropoff_longitude               -73.9818
payment_amount                       8.8
Name: 0, dtype: object
10906858


In [63]:
import State as State
importlib.reload(State)        

class MarkovChain:
    # num centers are we picking for k-means
    def __init__(self, raw, k):
        self.state_set = set()
        self.id_to_state = {}
        self.adj_matrix = None
        self.raw = raw
        
        self.initialize_centers(k)
        epsilon = 1e-6
        self.build_states_kmeans(1001, epsilon)
        
        self.add_points_edges()
        self.make_adjacency_matrix()
    
    def initialize_centers(self, k):
        ind = [i for i in range(len(self.raw))]
        rand.shuffle(ind)
        centers = ind[:k]
        # initialize centers
        ident = 0
        for c_ind in centers:
            # out of convenience, we aren't messing with pickup lat lon
            lat = self.raw.ix[c_ind]["dropoff_latitude"]
            lon = self.raw.ix[c_ind]["dropoff_longitude"]
            s = State.State((lat, lon), ident)
            self.state_set.add(s)
            self.id_to_state[ident] = s
            ident += 1
    
    def build_states_kmeans(self, iterations, epsilon):
        # run kmeans algorithm
        min_diff = 1e6
        while iterations > 0 and min_diff > epsilon:
            for ind, row in self.raw.iterrows():
                pos_start, pos_end = self.row_to_positions(row)
                closest_to_start = self.find_closest_state(pos_start)
                closest_to_end = self.find_closest_state(pos_end)
                
                closest_to_start.add_position(pos_start)
                closest_to_end.add_position(pos_end)
            max_diff = 0
            for s in self.state_set:
                max_diff = max(max_diff, s.update_center())
            min_diff = min(min_diff, max_diff)
            if iterations % 10 == 0:
                print(iterations)
            iterations -= 1 

    def add_points_edges(self):
        for s in self.state_set:
            s.clear_stored_data()
        for ind, row in self.raw.iterrows():
            pos_start, pos_end = self.row_to_positions(row)
            closest_to_start = self.find_closest_state(pos_start)
            closest_to_end = self.find_closest_state(pos_end)
            
            fare = self.row_to_fare(row)
            tdistance = self.row_to_distance(row)
            
            #Add points to respective states
            closest_to_start.store_data(pos_start)
            closest_to_end.store_data(pos_end)
            
            ##Add this edge to markov state
            closest_to_start.add_destination(closest_to_end.id, fare, tdistance)
    
    
    def make_adjacency_matrix(self):
        self.adj_matrix = np.ndarray(shape=(len(self.state_set), len(self.state_set)), dtype=float, order='C')
        for i in sorted(self.id_to_state.keys()):
            for j in sorted(self.id_to_state.keys()):
                self.adj_matrix[i][j] = self.transition_probability(i, j)
    
    def sum_of_square_error(self):
        total = 0
        for s in self.state_set:
            total += s.sum_of_squared_errors
        return total
    
    
#     def random_walk_given_time_cap(self, start_id, duration_cap):
#         while 
    
    def random_walk(self, start_id, walk_length):
        total_duration = 0
        total_fare = 0
        states_visited = []
        next_id = start_id
        for i in range(walk_length):
            states_visited.append(next_id)
            s = self.get_state(next_id)
            next_id, fare, duration = s.next_state()
            total_fare += fare
            total_duration += duration
        return states_visited, total_fare, total_duration
            
    def traveling_salesman(self, start_id):
        total_duration = 0
        total_fare = 0
        states_visited = []
        next_id = start_id
        need_to_visit = set(self.id_to_state.keys()[:])
        need_to_visit.remove(start_id)
        while(len(need_to_visit)):
            states_visited.append(next_id)
            s = self.get_state(next_id)
            next_id, fare, duration = s.next_state()
            total_fare += fare
            total_duration += duration
            need_to_visit.remove(next_id)
        return states_visited, total_fare, total_duration
    
    ##
    # GETTERS
    ##
    
    def get_state(self, iden):
        return self.id_to_state(iden)
    
    def get_adjacency_matrix(self):
        return self.adj_matrix
    
    ###
    # HELPER METHODS
    ###
    def find_closest_state(self, pos):
        def distance(state, pos):
            clat, clon = state.center
            return ((clat - pos[0])**2 + (clon - pos[1])**2)**0.5
        closest = None
        min_dist = None
        for state in self.state_set:
            d = distance(state, pos)
            if closest == None or d < min_dist:
                closest = state
                min_dist = d
        return closest
    
    def row_to_positions(self, row):
        lats = row["pickup_latitude"]
        lons = row["pickup_longitude"]
        pos_start = (lats, lons)

        late = row["dropoff_latitude"]
        lone = row["dropoff_longitude"]
        pos_end = (late, lone)
        
        return pos_start, pos_end
    
    def row_to_fare(self, row):
        return row["payment_amount"]
    def row_to_distance(self, row):
        return row["trip_distance"]
    
    def transition_probability(self, i, j):
        return self.id_to_state[i].probability_to(j)


In [45]:
def find_optimal_k(raw, k_list):
    y = []
    x = []
    k_dict = {}
    for t in trials:
        for k in k_list:
            if k not in k_dict:
                k_dict[k] = []
            try:
                m = MarkovChain(raw[:1000], k)
                x.append(k)
                y.append(m.sum_of_square_error())
            except ZeroDivisionError:
                print("hello")
    return x, y

In [46]:
x, y = find_optimal_k(raw, k_list=[i for i in range(5, 50)])

NameError: name 'trials' is not defined

In [47]:
plt.plot(x, y)

NameError: name 'x' is not defined

In [64]:
m = MarkovChain(raw[:1000], 10)

1000
990


In [65]:
adjm = m.get_adjacency_matrix()

In [66]:
np.sum(adjm, axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [67]:
S, U = np.linalg.eig(adjm.T)

In [68]:
print(U.T)

[[ -1.22474838e-02+0.j          -5.29523566e-01+0.j          -3.67304611e-01+0.j
   -4.15027273e-01+0.j          -1.90925611e-01+0.j          -8.27654471e-03+0.j
   -2.96526437e-01+0.j          -1.58744233e-01+0.j          -4.06206743e-01+0.j
   -3.12476896e-01+0.j        ]
 [ -9.25998211e-01+0.j           1.09865046e-01+0.j           1.34589898e-01+0.j
    2.70453124e-01+0.j           8.36409110e-02+0.j           4.21987384e-03+0.j
    1.25914861e-01+0.j           8.73900809e-02+0.j           9.13680795e-02+0.j
    1.85563365e-02+0.j        ]
 [  1.82153640e-03+0.j          -8.98383793e-02+0.j          -3.20158705e-02+0.j
    5.79522391e-01+0.j           1.76797301e-01+0.j          -5.86109614e-03+0.j
   -2.13654739e-01+0.j          -7.06169506e-01+0.j           2.81745646e-01+0.j
    7.65271675e-03+0.j        ]
 [ -4.30056775e-04+0.00235193j   9.38982408e-03-0.15346278j
   -3.34467726e-01-0.01226587j   6.01230927e-01+0.j
   -3.35162144e-01+0.05408494j  -1.30479250e-02+0.00182967j
   

In [69]:
# U is row major
inv_dist = U.T[0]
print(inv_dist)

[-0.01224748+0.j -0.52952357+0.j -0.36730461+0.j -0.41502727+0.j
 -0.19092561+0.j -0.00827654+0.j -0.29652644+0.j -0.15874423+0.j
 -0.40620674+0.j -0.31247690+0.j]


In [70]:
norm_inv_dist = inv_dist / float(sum(inv_dist))

/Users/cheng/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


In [71]:
inv2 = sorted(norm_inv_dist)

In [72]:
old_norm_inv_dist = sorted(norm_inv_dist)

In [73]:
old_norm_inv_dist

[(0.0030685015748821271-0j),
 (0.0045407141202921638-0j),
 (0.058853899313321544-0j),
 (0.070785038614818788-0j),
 (0.10993619559566692-0j),
 (0.11584977568799031-0j),
 (0.13617696956784739-0j),
 (0.15059980637930526-0j),
 (0.15386998865090754-0j),
 (0.19631911049496778-0j)]